In [1]:
! pip install datasets transformers evaluate -q
! pip install tensorflow
! pip install torch torchvision
! pip install tf-keras
! pip install scikit-learn
! pip install tqdm

In [2]:
from datasets import load_from_disk
from transformers import pipeline, AutoTokenizer
import tqdm.auto as tqdm

# Read the BOLD JSON files into a dataset
load_path='./prompts/sample_bold_prompts'
bold_prompts = load_from_disk(load_path)

# Load text generation pipeline and model
model = "gpt2"
text_generation = pipeline("text-generation", model=model)
tokenizer = AutoTokenizer.from_pretrained(model)

/Users/Connie/Documents/sandbox/BOLD-LLM-Bias/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-12 13:59:01.245674: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def generate_text(prompts, max_length=100, do_sample=False, pad_token_id=502256):
	model_continuations=[]
	# Iterate through prompts and generate text
	for prompt in tqdm.tqdm(prompts):
		generation = text_generation(prompt,
									 max_length=max_length,
									 do_sample=do_sample,
									 pad_token_id=pad_token_id,
									 truncation=True)
		continuation = generation[0]['generated_text'].replace(prompt,'').lstrip()
		model_continuations.append(continuation)
	return model_continuations

In [4]:
# Select only first instance of prompt, some prompts have multiple sentences
toxic_prompts = [p['prompts'] for p in bold_prompts]

# Generate text with LLM using prompts
model_continuations = generate_text(toxic_prompts)

print('Generated '+ str(len(model_continuations))+ ' continuations')

# Update dataset with generations
bold_prompts = bold_prompts.add_column(f'generated_prompts_{model}', model_continuations)

100%|██████████| 100/100 [07:23<00:00,  4.43s/it]


Generated 100 continuations


In [5]:
# Save resulting prompts to disk
out_name = f'./generated_data/prompts_{model}'
bold_prompts.save_to_disk(out_name)

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 2554.29 examples/s]
